In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:

train = pd.read_csv("../input/hackerearth-ml-challenge-pet-adoption/train.csv")
test=pd.read_csv("../input/hackerearth-ml-challenge-pet-adoption/test.csv")

In [ ]:
train.shape,test.shape

In [ ]:
train.info()

In [ ]:
test.info()

In [ ]:
train['breed_category'].value_counts()

In [ ]:
a=train['breed_category'][(np.isnan(train['condition']))]
a.value_counts()

In [ ]:
test_id=test['pet_id']
ntrain=train.shape[0]

In [ ]:
y1=train['breed_category']
y2=train['pet_category']

In [ ]:
all_data = pd.concat((train, test)).reset_index(drop=True)
all_data.drop(['breed_category','pet_category'], axis=1, inplace=True)

In [ ]:
all_data['condition'].value_counts()

In [ ]:
all_data['condition'].fillna(-1,inplace=True)


In [ ]:
all_data['condition'].value_counts()

In [ ]:
all_data['issue_date']=pd.to_datetime(all_data['issue_date'])
all_data['listing_date']=pd.to_datetime(all_data['listing_date'])

In [ ]:
x=[]
for d in all_data['issue_date']:
    y=d.month
    x.append(y)
all_data['issue_month']=x

In [ ]:
x=[]
for d in all_data['listing_date']:
    y=d.month
    x.append(y)
all_data['listing_month']=x

In [ ]:
x=[]
for d in all_data['listing_date']:
    y=d.year+(d.month/12.0)+(d.day/365.0)
    x.append(y)
all_data['modified_listing_date']=x

In [ ]:
x=[]
for d in all_data['issue_date']:
    y=d.year+(d.month/12.0)+(d.day/365.0)
    x.append(y)
all_data['modified_issue_date']=x

In [ ]:
all_data['took_time']=abs(all_data['modified_listing_date']-all_data['modified_issue_date'])


In [ ]:
all_data['1stnum'] = all_data['pet_id'].str[:6]
all_data['1st2num'] = all_data['pet_id'].str[:7]

In [ ]:
train = all_data[:ntrain]
test = all_data[ntrain:]

In [ ]:
x=train.drop(['pet_id','issue_date','listing_date','modified_issue_date'],axis=1)
test=test.drop(['pet_id','issue_date','listing_date','modified_issue_date'],axis=1)

In [ ]:
x.select_dtypes(exclude='number').columns.to_list()

In [ ]:
x=pd.get_dummies(x)
test=pd.get_dummies(test)

In [ ]:
x.shape,test.shape

In [ ]:
a=set(x.columns)-set(test.columns)
a=list(a)
a

In [ ]:
x=x.drop(a,axis=1)

In [ ]:
x.shape,test.shape

In [ ]:
all_data = pd.concat((x, test)).reset_index(drop=True)

In [ ]:
from sklearn import preprocessing

names = all_data.columns

scaler = preprocessing.StandardScaler()

scaled_df = scaler.fit_transform(all_data)
all_data = pd.DataFrame(scaled_df, columns=names)

In [ ]:
x = all_data[:ntrain]
test = all_data[ntrain:]

In [ ]:
from sklearn.model_selection import train_test_split
x1_train,x1_test,y1_train,y1_test=train_test_split(x,y2,test_size=0.2,random_state=44,shuffle=True)

In [ ]:
from xgboost import XGBClassifier

In [ ]:
model1 = XGBClassifier()
model1.fit(x1_train, y1_train)

In [ ]:
new_feat=model1.predict(x)
#output1 is new first output i.e the predicted pet_category of model 1 for test data
output1=model1.predict(test)
#vld1 is validation 1 i.e we'll check score with the predicted result of validation data of model 1
vld1=model1.predict(x1_test)

In [ ]:
x2 = pd.DataFrame(x, columns=names)
test2 = pd.DataFrame(test, columns=names)

In [ ]:
x2['output1']=new_feat
#the predicted pet_category of model 1 for test data is used as a input variable or feature of the test data of model 2
test2['output1']=output1

In [ ]:
x2_train,x2_test,y2_train,y2_test=train_test_split(x2,y1,test_size=0.2,random_state=44)
xgb2 = XGBClassifier()
xgb2=xgb2.fit(x2_train,y2_train)

In [ ]:
output2=xgb2.predict(test)
#vld2 is validation 2 i.e we'll check score with the predicted result of validation data of model 2
vld2=xgb2.predict(x2_test)

In [ ]:
from sklearn.metrics  import f1_score

In [ ]:
s1=f1_score(y1_test,vld1,average='weighted')
s2=f1_score(y2_test,vld2,average='weighted')
accuracy=100*((s1+s2)/2)
accuracy

In [ ]:
sub_new=pd.DataFrame({
    "pet_id":test_id,
    "breed_category":output2,
    "pet_category":output1
})
sub_new.to_csv("sub_pet.csv",index=False)